In [1]:
import win32com.client
import pandas as pd
import datetime

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [21]:
inbox = outlook.GetDefaultFolder(6) #6 is the inbox, can change it for other folders

In [22]:
#see all the folders that are available
for folder in outlook.Folders: 
    print(folder.Name)

Public Folders - isabel.fernandez@maastrichtuniversity.nl
assistant-iberialatam@maastrichtuniversity.nl
Public Folders - info-iberialatam@maastrichtuniversity.nl
Public Folders - assistant-iberialatam@maastrichtuniversity.nl
info-iberialatam@maastrichtuniversity.nl
Outlook Data File
i.fernandezpereira@student.maastrichtuniversity.nl
Public Folders - i.fernandezpereira@student.maastrichtuniversity.nl
isabel.fernandez@maastrichtuniversity.nl


In [25]:
if 1==0: #make this true when you want the main inbox
    messages = inbox.Items
    
else: #use this when trying to access another folder or a delegated mailbox
    
    root_folder = outlook.Folders.Item(2) #this allows you to access different inboxes 
    #make sure correct inbox is being used
    print(root_folder)
    subfolder = root_folder.Folders['Inbox']
    #get emails from that inbox
    messages = subfolder.Items
    
#sort messages by the time they were received
messages.Sort("ReceivedTime")
message = messages.GetFirst()
print(message)

#attach = message.Attachments
#print (body_content)
#print(datetime)
#print(sender)
#print(subject)

assistant-iberialatam@maastrichtuniversity.nl
Re: Co-worker for Greece/Cyprus


In [16]:
#create empty dataframe, fill it in as emails are read
df= pd.DataFrame(columns=['subject', 'sender', 'date', 'content of message'])

In [17]:
#first cast the datetime to string, so that it can be stripped of the timezone and used more easily
#The error correction can propably be improved
def get_date (date, i):
    date= str(date)
    try:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M:%S')
    except ValueError:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M')
    df.loc[i,'date'] =date
#read body firt, remove unwanted text, then feed it to the df
def get_content (body, i):
    body= body.replace("\r"," ") 
    body= body.replace("\n","") 
    body= body.replace("\t","") 
    df.loc[i,'content of message'] = body

In [18]:
for i in range(0,50):
    try:
        df.loc[i,'subject']=message.Subject
        df.loc[i,'sender']=message.Sender
        get_date(message.CreationTime, i)      
        get_content(message.body, i)
        
    except AttributeError as e:
        if e == 'GetNext.Subject':
            df.loc[i,'subject']= None
            df.loc[i,'sender'] = message.Sender
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e == 'GetNext.Sender':
            df.loc[i,'sender'] = None
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e=='GetNext.message.body':
            df.loc[i,'content of message'] = None
           
            
    message= messages.GetNext()



In [19]:
display(df)

,subject,sender,date,content of message
0,Re: Co-worker for Greece/Cyprus,"Iliopoulos, Panos (Stud. FASoS)",2017-10-16 09:08:47,"​​Hi Konrad, Will let you know as soon as..."
1,Meeting Forward Notification: Strategic Meetin...,NaN,NaN,NaN
2,Meeting Forward Notification: Strategic Meetin...,NaN,NaN,NaN
3,Accepted: Strategic Meeting Germany,NaN,NaN,NaN
4,News overview - week 41,"Pers, (BU)",2017-10-17 11:24:04,"Hello, With this email you find the news o..."
5,Update attendance UK & Germany,"Weerts, Aisha (BU)",2017-10-17 11:40:01,"Hi Guys, Could you provide me with a detai..."
6,FW: Draft agenda strategic meeting,"Boer, Zoe den (BU)",2017-10-17 13:52:45,"Hoi Mien, Ben je akkoord met de agenda voo..."
7,Accepted: Strategic Meeting Germany,NaN,NaN,NaN
8,Re: Co-worker for Greece/Cyprus,"Iliopoulos, Panos (Stud. FASoS)",2017-10-17 14:45:32,"Hi Konrad, Had the CRM training this morn..."
9,RE: Study in Holland Webinars - October 2017,"Boer, Zoe den (BU)",2017-10-17 15:19:48,"Hi Konrad, Did you have a look at below? ..."


In [20]:
#change file name accordingly 
df.to_csv('.\dataset\Delegated_Email.csv')

In [27]:
def num_question(email):
    counter=0
    for word in email:
        if word == '?':
            counter+=1
    
    return counter
        

In [46]:
string =df.loc[5,'content of message']
print(string)

Hi Guys,     Could you provide me with a detailed list of who’s accepted the invitation to CT Germany & UK & Ireland?  Thanks!     Met vriendelijke groet,  With kind regards,       Aisha Weerts  International Recruitment Advisor     aisha.weerts@maastrichtuniversity.nl <mailto:aisha.weerts@maastrichtuniversity.nl>   www.maastrichtuniversity.nl <http://www.maastrichtuniversity.nl/>      Minderbroedersberg 4-6, 6211 LK Maastricht, The Netherlands | Room 4.027  P.O. Box 616, 6200 MD Maastricht, The Netherlands     T +31 (0)43 38 85 226       facebook.com/maastrichtuniversity <https://nl-nl.facebook.com/maastricht.university/>     youtube.com/maastrichtuniversity <https://www.youtube.com/user/maastrichtuniversity>     twitter.com/maastrichtuniversity <https://twitter.com/maastrichtu>        


In [47]:
num_question(string)

1